In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
lyrics = pd.read_csv(os.pardir+'/metadata_plus/Song_Lyrics.csv')
songs = pd.read_csv(os.pardir+'/metadata_plus/Song.csv')
stations = pd.read_csv(os.pardir+'/metadata_plus/Station_Song.csv')
station_tags = pd.read_csv(os.pardir+'/metadata_plus/Station_Tag.csv')
tag_info = pd.read_csv(os.pardir+'/metadata_plus/Station_Tag_Info.csv')

In [3]:
lyrics = lyrics.fillna('')

In [4]:
tag_info

,IDX,TYPE_ID,TAG_NAME,CDATE
0,5322,2,화나요,2021-06-08 12:07:54
1,5330,2,행복해요,2021-06-08 12:07:54
2,5711,2,편안해요,2022-03-17 14:48:33
3,5759,12,슬픔,2022-03-31 14:31:33
4,5762,12,불안,2022-03-31 14:31:33
5,5766,12,신남,2022-03-31 14:31:33


#### 가사 정보에 대한 결측치 확인

In [5]:
length = lyrics['LYRICS'].fillna('').apply(lambda x:len(x))
length.quantile(0.07)

2.080000000000041

In [6]:
# 가사의 길이가 100자 이하라면 가사정보가 누락된 것으로 추정한다.
lyrics[length < 100].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279 entries, 4 to 3744
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   IDX      279 non-null    int64 
 1   SONG_ID  279 non-null    int64 
 2   LYRICS   279 non-null    object
dtypes: int64(2), object(1)
memory usage: 8.7+ KB


#### 각 스테이션의 구성 확인

- 57개의 스테이션, 내부에 수록된 곡의 수는 변동
- 각 스테이션마다 6개의 세부 감성으로 분류된 태깅 존재

In [7]:
stations['ST_ID'].unique().shape

(221,)

In [10]:
tag_dict = {}
for i in range(len(tag_info)):
    tag_dict[tag_info['IDX'].iloc[i]] = tag_info['TAG_NAME'].iloc[i]

In [11]:
station_tag_dict = {}
for i in range(len(station_tags)):
    station_tag_dict[station_tags['ST_ID'].iloc[i]] = station_tags['ST_TAG_ID'].iloc[i]

In [12]:
song_tag_dict = {}
for i in range(len(songs)):
    song_tag_dict[songs['SONG_ID'].iloc[i]] = songs['SONG_TITLE'].iloc[i]

In [13]:
lyrics_dict = {}
for i in range(len(lyrics)):
    songid = lyrics['SONG_ID'].iloc[i]
    lyric = lyrics['LYRICS'].iloc[i]
    if len(lyric) < 100:
        lyrics_dict[songid] = 'NOLYRICS'
    else:
        lyrics_dict[songid] = lyric

- 스테이션에 포함된 곡들을 해당 스테이션의 감성으로 태깅
- song_id, song_name, song_lyrics, senti_tag 로 구성

In [23]:
station_ids = stations['ST_ID'].unique()

song_id = []
song_name = []
song_lyric = []
senti_tag = []

for case in station_ids:
    temp = stations[stations['ST_ID'] == case]
    for i in range(len(temp)):
        try:
            senti = tag_dict[station_tag_dict[temp['ST_ID'].iloc[i]]]
            s_id = temp['SONG_ID'].iloc[i]
            s_name = song_tag_dict[temp['SONG_ID'].iloc[i]]
            s_lyric = lyrics_dict[temp['SONG_ID'].iloc[i]]
            
            senti_tag.append(senti)
            song_id.append(s_id)
            song_name.append(s_name)
            song_lyric.append(s_lyric)
        except:
            pass
        
        
    

In [24]:
print(len(song_id), len(song_name), len(song_lyric), len(senti_tag))

4916 4916 4916 4916


In [25]:
result = pd.DataFrame(
    data=[song_id, song_name, song_lyric, senti_tag]
).T
result.columns= ['song_id', 'song_name', 'song_lyric', 'senti_tag']

In [26]:
result

,song_id,song_name,song_lyric,senti_tag
0,2466,오늘도 난,나도 알지 못한날 굳이 알려고 들지마 \n보여줄 수 없는 내 마음만 안타까울 뿐 \...,편안해요
1,33732,동반자,사랑이기엔 우매했던\n긴 시간의 끝이\n어느덧 처음 만난 그때처럼\n내겐 아득하오\...,편안해요
2,111208,사랑한다는 말,처음으로 사랑한다 말하던 날\n살며시 농담처럼 흘리던 말\n못 알아들은 걸까\n딴청...,편안해요
3,111216,망각,지금의 그와 혹\n내 얘기가 나올 때\n사랑했다고\n솔직히 말을 하는지\n아니면 그...,편안해요
4,421755,너에게,나의 하늘을 본 적이 있을까\n조각 구름과 빛나는 별들이\n끝없이 펼쳐있는\n구석진...,편안해요
...,...,...,...,...
4911,34794445,Songcry,NOLYRICS,불안
4912,34797285,자가격리,우린 스스로 될 필요가 있지 자가격리\n며칠 간의 호텔 예약했어 no covid\n...,불안
4913,34800760,Gucci Bandanna (feat. lobonabeat!),Gucci Bandanna\nGucci Gucci Bandanna\nGucci Ba...,불안
4914,34800774,"Refresh (Feat. OLNL, B JYUN.)",돌아가 보자고 Refresh\n좋았던 때로\n돌아가 보자고 Refresh\n혼자일 ...,불안


In [28]:
result.to_csv('Raw_data_plus.csv', index=False)

In [20]:
stations[stations['ST_ID'] == station_ids[0]]

,IDX,ST_ID,SONG_ID,PRIORITY
0,10669994,467710,2466,2
32,10669997,467710,33732,5
113,10669998,467710,111208,6
115,10669999,467710,111216,7
148,10670003,467710,421755,11
193,10670005,467710,526820,13
316,10670000,467710,1590175,8
423,10669995,467710,2043348,3
953,10669996,467710,4142913,4
975,10669993,467710,4191056,1
